<a href="https://colab.research.google.com/github/noguchisatoshi/for_public/blob/main/FEM_tmp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://qiita.com/Altaka4128/items/41101c96729b68d7c96f

### Some modules

In [ ]:
# 3次元節点格納用のクラス
class Node:

    # コンストラクタ
    # no : 節点番号
    # x  : x座標
    # y  : y座標
    # z  : z座標
    def __init__(self, no, x, y, z):
        self.no = no   # 節点番号
        self.x = x     # x座標
        self.y = y     # y座標
        self.z = z     # z座標

    # 節点の情報を表示する
    def printNode(self):
        print("Node No: %d, x: %f, y: %f, z: %f" % (self.no, self.x, self.y, self.z))

In [ ]:
import numpy as np

class Dmatrix:
    # コンストラクタ
    # young   : ヤング率
    # poisson : ポアソン比
    def __init__(self, young, poisson):
        self.young = young
        self.poisson = poisson

    # 弾性状態のDマトリクスを作成する
    def makeDematrix(self):

        tmp = self.young / ((1.0 + self.poisson) * (1.0 - 2.0 * self.poisson))
        matD = np.array([[1.0 - self.poisson, self.poisson, self.poisson, 0.0, 0.0, 0.0],
                         [self.poisson, 1.0 - self.poisson, self.poisson, 0.0, 0.0, 0.0],
                         [self.poisson, self.poisson, 1.0 - self.poisson, 0.0, 0.0, 0.0],
                         [0.0, 0.0, 0.0, 0.5 * (1.0 - 2.0 * self.poisson), 0.0, 0.0],
                         [0.0, 0.0, 0.0, 0.0, 0.5 * (1.0 - 2.0 * self.poisson), 0.0],
                         [0.0, 0.0, 0.0, 0.0, 0.0, 0.5 * (1.0 - 2.0 * self.poisson)]])
        matD = tmp * matD

        return matD

In [ ]:
import numpy as np
import numpy.linalg as LA
#from Dmatrix import Dmatrix
#from Node import Node

# 四面体4節点要素のクラス
class C3D4:
    # コンストラクタ
    # no           : 要素番号
    # nodes        : 節点の集合(Node型のリスト)
    # young        : ヤング率
    # poisson      : ポアソン比
    # density      : 密度
    # vecGravity   : 重力加速度のベクトル(np.array型)
    def __init__(self, no, nodes, young, poisson, density, vecGravity = None):

        # インスタンス変数を定義する
        self.nodeNum = 4               # 節点の数
        self.nodeDof = 3               # 節点の自由度
        self.no = no                   # 要素番号
        self.nodes = nodes             # nodesは反時計回りの順番になっている前提(Node2d型のリスト形式)
        self.young = young             # ヤング率
        self.poisson = poisson         # ポアソン比
        self.density = density         # 密度
        self.vecGravity = vecGravity   # 重力加速度のベクトル(np.array型)
        self.ipNum = 1                 # 積分点の数
        self.w = 1.0 / 6.0             # 積分点の重み係数
        self.ai = 1.0 / 4.0            # 積分点の座標(a,b,c座標系)
        self.bi = 1.0 / 4.0            # 積分点の座標(a,b,c座標系)
        self.ci = 1.0 / 4.0            # 積分点の座標(a,b,c座標系)

    # 要素剛性マトリクスKeを作成する
    def makeKematrix(self):

        # ヤコビ行列を計算する
        matJ = self.makeJmatrix()

        # Bマトリクスを計算する
        matB = self.makeBmatrix()

        # Dマトリクスを計算する
        matD = self.makeDmatrix()

        # Ketマトリクスをガウス積分で計算する
        matKet = self.w * matB.T @ matD @ matB * LA.det(matJ)

        return matKet

    # Dマトリクスを作成する
    def makeDmatrix(self):

        matD = Dmatrix(self.young, self.poisson).makeDematrix()
        return matD

    # ヤコビ行列を計算する
    def makeJmatrix(self):

        dxda = -self.nodes[0].x + self.nodes[1].x
        dyda = -self.nodes[0].y + self.nodes[1].y
        dzda = -self.nodes[0].z + self.nodes[1].z
        dxdb = -self.nodes[0].x + self.nodes[2].x
        dydb = -self.nodes[0].y + self.nodes[2].y
        dzdb = -self.nodes[0].z + self.nodes[2].z
        dxdc = -self.nodes[0].x + self.nodes[3].x
        dydc = -self.nodes[0].y + self.nodes[3].y
        dzdc = -self.nodes[0].z + self.nodes[3].z

        matJ = np.array([[dxda, dyda, dzda],
                         [dxdb, dydb, dzdb],
                         [dxdc, dydc, dzdc]])        

        # ヤコビアンが負にならないかチェックする
        if LA.det(matJ) < 0:
            raise ValueError("要素の計算に失敗しました")

        return matJ

    # Bマトリクスを作成する
    def makeBmatrix(self):

        # dNi/da, dNi/dbを計算する
        dN1da = -1.0
        dN2da = 1.0
        dN3da = 0.0
        dN4da = 0.0
        dN1db = -1.0
        dN2db = 0.0
        dN3db = 1.0
        dN4db = 0.0
        dN1dc = -1.0
        dN2dc = 0.0
        dN3dc = 0.0
        dN4dc = 1.0

        # dNi/dx, dNi/dyを計算する
        matdNdab = np.matrix([[dN1da, dN2da, dN3da, dN4da],
                              [dN1db, dN2db, dN3db, dN4db],
                              [dN1dc, dN2dc, dN3dc, dN4dc]])

         # ヤコビ行列を計算する
        matJ = self.makeJmatrix()

        #dNdxy = matJinv * matdNdab
        dNdxy = LA.solve(matJ, matdNdab)

        # Bマトリクスを計算する
        matB = np.array([[dNdxy[0, 0], 0.0, 0.0, dNdxy[0, 1], 0.0, 0.0, dNdxy[0, 2], 0.0, 0.0, dNdxy[0, 3], 0.0, 0.0],
                         [0.0, dNdxy[1, 0], 0.0, 0.0, dNdxy[1, 1], 0.0, 0.0, dNdxy[1, 2], 0.0, 0.0, dNdxy[1, 3], 0.0],
                         [0.0, 0.0, dNdxy[2, 0], 0.0, 0.0, dNdxy[2, 1], 0.0, 0.0, dNdxy[2, 2], 0.0, 0.0, dNdxy[2, 3]],
                         [0.0, dNdxy[2, 0], dNdxy[1, 0], 0.0, dNdxy[2, 1], dNdxy[1, 1], 0.0, dNdxy[2, 2], dNdxy[1, 2], 0.0, dNdxy[2, 3], dNdxy[1, 3]],
                         [dNdxy[2, 0], 0.0, dNdxy[0, 0], dNdxy[2, 1], 0.0, dNdxy[0, 1], dNdxy[2, 2], 0.0, dNdxy[0, 2], dNdxy[2, 3], 0.0, dNdxy[0, 3]],
                         [dNdxy[1, 0], dNdxy[0, 0], 0.0, dNdxy[1, 1], dNdxy[0, 1], 0.0, dNdxy[1, 2], dNdxy[0, 2], 0.0, dNdxy[1, 3], dNdxy[0, 3], 0.0]])

        return matB

    # 等価節点力の荷重ベクトルを作成する
    def makeEqNodeForceVector(self):

        # ヤコビ行列を計算する
        matJ = self.makeJmatrix()

        vecEqNodeForce = np.zeros(self.nodeNum * self.nodeDof)

        # 物体力による等価節点力を計算する
        vecBodyForce = np.zeros(self.nodeNum * self.nodeDof)
        if not self.vecGravity is None:
            vecb = self.density * self.vecGravity   # 単位体積あたりの物体力のベクトル
            N1 = 1 - self.ai - self.bi - self.ci
            N2 = self.ai
            N3 = self.bi
            N4 = self.ci
            matN = np.matrix([[N1, 0.0, 0.0, N2, 0.0, 0.0, N3, 0.0, 0.0, N4, 0.0, 0.0],
                              [0.0, N1, 0.0, 0.0, N2, 0.0, 0.0, N3, 0.0, 0.0, N4, 0.0],
                              [0.0, 0.0, N1, 0.0, 0.0, N2, 0.0, 0.0, N3, 0.0, 0.0, N4]])
            vecBodyForce = self.w * np.array(matN.T @ vecb).flatten() * LA.det(matJ)

        # 表面力と物体力の等価節点力を計算する
        vecEqNodeForce = vecBodyForce

        return vecEqNodeForce

In [ ]:
import numpy as np

# 境界条件を格納するクラス
class Boundary:
    # コンストラクタ
    # nodeNum : 節点数
    def __init__(self, nodeNum):
        # インスタンス変数を定義する
        self.nodeNum = nodeNum                                     # 全節点数
        self.nodeDof = 3                                           # 節点の自由度
        self.vecDisp = np.array(nodeNum * self.nodeDof * [None])   # 単点拘束の強制変位
        self.vecForce = np.array(nodeNum * self.nodeDof * [0.0])   # 荷重ベクトル
        self.matC = np.empty((0, nodeNum * self.nodeDof))          # 多点拘束用のCマトリクス
        self.vecd = np.empty(0)                                    # 多点拘束用のdベクトル

    # 単点拘束を追加する
    # nodeNo : 節点番号
    # dispX  : x方向の強制変位
    # dispY  : y方向の強制変位
    # dispZ  : z方向の強制変位
    def addSPC(self, nodeNo, dispX, dispY, dispZ):

        self.vecDisp[self.nodeDof * (nodeNo - 1) + 0] = dispX
        self.vecDisp[self.nodeDof * (nodeNo - 1) + 1] = dispY
        self.vecDisp[self.nodeDof * (nodeNo - 1) + 2] = dispZ

    # 多点拘束を追加する
    # 条件式 : vecC x u = d
    def addMPC(self, vecC, d):

        self.matC = np.vstack((self.matC, vecC))
        self.vecd = np.hstack((self.vecd, d))

    # 単点拘束条件から変位ベクトルを作成する
    def makeDispVector(self):

        return self.vecDisp

    # 荷重を追加する
    def addForce(self, nodeNo, fx, fy, fz):

        self.vecForce[self.nodeDof * (nodeNo - 1) + 0] = fx
        self.vecForce[self.nodeDof * (nodeNo - 1) + 1] = fy
        self.vecForce[self.nodeDof * (nodeNo - 1) + 2] = fz

    # 境界条件から荷重ベクトルを作成する
    def makeForceVector(self):

        return self.vecForce

    # 多点拘束の境界条件を表すCマトリクス、dベクトルを作成する
    def makeMPCmatrixes(self):

        return self.matC, self.vecd

    # 拘束条件を出力する
    def printBoundary(self):
        print("Node Number: ", self.nodeNum)
        print("SPC Constraint Condition")
        print(self.vecDisp)
        print("Force Condition")
        print(self.vecForce)
        print("MPC Constraint Condition")
        print("C x u = d")
        print("C Matrix")
        print(self.matC)
        print("d vector")
        print(self.vecd)


In [ ]:
import numpy as np
import numpy.linalg as LA
#from Boundary import Boundary

class FEM:
    # コンストラクタ
    # nodes    : 節点は1から始まる順番で並んでいる前提(Node型のリスト)
    # elements : 要素は種類ごとにソートされている前提(C3D4型のリスト)
    # bound    : 境界条件(d2Boundary型)
    def __init__(self, nodes, elements, nodes_head, elements_head, bound):

        # インスタンス変数を定義する
        self.nodeDof = 3   # 節点の自由度
        self.nodes = nodes
        self.elements = elements
        self.bound = bound
        
        self.nodes_head = nodes_head
        self.elements_head = elements_head
        
        self.vecStrain= np.array(len(self.elements) * self.nodeDof * 2 * [None]) # For strain tensor
        self.vecStress= np.array(len(self.elements) * self.nodeDof * 2 * [None]) # For stress tensor
 
    # 解析を行う
    def analysis(self):

        # 境界条件を考慮しないKマトリクスを作成する
        matK = self.makeKmatrix()

        # 荷重ベクトルを作成する
        vecf = self.makeForceVector()

        # 境界条件を考慮したKマトリクス、荷重ベクトルを作成する
        matKc, vecfc, mpcNum = self.setBoundCondition(matK, vecf)

        if np.isclose(LA.det(matKc), 0.0) :
            raise ValueError("有限要素法の計算に失敗しました。")

        # 変位ベクトルを計算する
        vecDisp = LA.solve(matKc, vecfc)
        self.vecDisp = vecDisp
        
        # ひずみテンソルを計算する
        vecStrain = self.calculateStrain()
        self.vecStrain = vecStrain
        
        # 応力テンソルを計算する
        vecStress = self.calculateStress()
        self.vecStress = vecStress

        # 節点反力を計算する
        vecRF = np.array(matK @ vecDisp - vecf).flatten()
        self.vecRF = vecRF

        return vecDisp, vecRF

    # 節点に負荷する荷重、等価節点力を考慮した荷重ベクトルを作成する
    def makeForceVector(self):

        # 節点に負荷する荷重ベクトルを作成する
        vecCondiForce = self.bound.makeForceVector()

        # 等価節点力の荷重ベクトルを作成する
        vecEqNodeForce = np.zeros(len(self.nodes) * self.nodeDof)
        for elem in self.elements:
            vecElemEqNodeForce = elem.makeEqNodeForceVector()
            for i in range(len(elem.nodes)):
                for j in range(self.nodeDof):
                    vecEqNodeForce[self.nodeDof * (elem.nodes[i].no - 1) + j] += vecElemEqNodeForce[self.nodeDof * i + j]

        # 境界条件、等価節点力の荷重ベクトルを足し合わせる
        vecf = vecCondiForce + vecEqNodeForce

        return vecf

    # 境界条件を考慮しないKマトリクスを作成する
    def makeKmatrix(self):

        matK = np.matrix(np.zeros((len(self.nodes) * self.nodeDof, len(self.nodes) * self.nodeDof)))
        for elem in self.elements:

            # ketマトリクスを計算する
            matKe = elem.makeKematrix()

            # Ktマトリクスに代入する
            for c in range(len(elem.nodes) * self.nodeDof):
                ct = (elem.nodes[c // self.nodeDof].no - 1) * self.nodeDof + c % self.nodeDof
                for r in range(len(elem.nodes) * self.nodeDof):
                    rt = (elem.nodes[r // self.nodeDof].no - 1) * self.nodeDof + r % self.nodeDof
                    matK[ct, rt] += matKe[c, r]

        return matK

    # Kマトリクス、荷重ベクトルに境界条件を考慮する
    # matK         : 剛性マトリクス
    # vecf         : 荷重ベクトル
    # vecBoundDisp : 節点の境界条件の変位ベクトル
    # vecDisp      : 全節点の変位ベクトル(np.array型)
    def setBoundCondition(self, matKt, vecf):

        matKtc = np.copy(matKt)
        vecfc = np.copy(vecf)
        vecBoundDisp = self.bound.makeDispVector()

        # 単点拘束条件を考慮したKマトリクス、荷重ベクトルを作成する
        for i in range(len(vecBoundDisp)):
            if not vecBoundDisp[i] == None:
                # Kマトリクスからi列を抽出する
                vecx = np.array(matKt[:, i]).flatten()

                # 変位ベクトルi列の影響を荷重ベクトルに適用する
                vecfc = vecfc - (vecBoundDisp[i]) * vecx

                # Kマトリクスのi行、i列を全て0にし、i行i列の値を1にする
                matKtc[:, i] = 0.0
                matKtc[i, :] = 0.0
                matKtc[i, i] = 1.0
        for i in range(len(vecBoundDisp)):
            if not vecBoundDisp[i] == None:
                vecfc[i] = vecBoundDisp[i]
                
        
        # 2023.3.16 追記
        # https://qiita.com/Altaka4128/items/003f0d0cc564175dca2c
        # 多節点拘束条件を考慮したKマトリクス、荷重ベクトルを作成する
        matC, vecd = self.bound.makeMPCmatrixes()
        mpcNum = len(vecd)   # 多節点拘束の条件式の数を求める
        
        if mpcNum > 0:
            matKc = np.hstack((matKc, matC.T))
            tmpMat = np.hstack((matC, np.zeros((matC.shape[0], matC.shape[0]))))
            matKc = np.vstack((matKc, tmpMat))
            vecfc = np.hstack((vecf, vecd))
        
        return matKtc, vecfc, mpcNum
    
    #追加 2023.3.15 for Strain
    def calculateStrain(self):
        vecStrain = np.copy(self.vecStrain)
        for i, elem in enumerate(self.elements):
            matB = elem.makeBmatrix()
            vecStrain_tmp = []
            for node in range(len(elem.nodes)):
                vecStrain_tmp.extend([self.vecDisp[self.nodeDof * node], self.vecDisp[self.nodeDof * node + 1], self.vecDisp[self.nodeDof * node + 2]])
            
            vecStrain_tmp = matB @ np.array(vecStrain_tmp).flatten()
            for j, comp in enumerate(vecStrain_tmp):
                vecStrain[i * self.nodeDof * 2 + j] = comp
        return vecStrain
    
    #追加 2023.3.15 for Stress
    def calculateStress(self):
        vecStress = np.copy(self.vecStress)
        for i, elem in enumerate(self.elements):
            matD = elem.makeDmatrix()
            vecStrain_tmp = [[self.vecStrain[self.nodeDof * i], self.vecStrain[self.nodeDof * i + 1], self.vecStrain[self.nodeDof * i + 2],
                                  self.vecStrain[self.nodeDof * i + 3], self.vecStrain[self.nodeDof * i + 4], self.vecStrain[self.nodeDof * i + 5]]]
            vecStress_tmp = matD @ np.array(vecStrain_tmp).flatten()
            for j, comp in enumerate(vecStress_tmp):
                vecStress[i * self.nodeDof * 2 + j] = comp
        return vecStress
        

    # 解析結果をテキストファイルに出力する
    def outputTxt(self, filePath):

        # ファイルを作成し、開く
        f = open(filePath + ".txt", 'w')

        # 出力する文字の情報を定義する
        columNum = 20
        floatDigits = ".10g"

        # 入力データのタイトルを書きこむ
        f.write("*********************************\n")
        f.write("*          Input Data           *\n")
        f.write("*********************************\n")
        f.write("\n")

        # 節点情報を出力する
        f.write("***** Node Data ******\n")
        f.write("No".rjust(columNum) + "X".rjust(columNum) + "Y".rjust(columNum) + "Z".rjust(columNum) + "\n")
        f.write("-" * columNum * 4 + "\n")
        for node in self.nodes:
            strNo = str(node.no).rjust(columNum)
            strX = str(format(node.x, floatDigits).rjust(columNum))
            strY = str(format(node.y, floatDigits).rjust(columNum))
            strZ = str(format(node.z, floatDigits).rjust(columNum))
            f.write(strNo + strX + strY + strZ + "\n")
        f.write("\n")

        # 要素情報を出力する
        nodeNoColumNum = 36
        f.write("***** Element Data ******\n")
        f.write("No".rjust(columNum) + "Type".rjust(columNum) + "Node No".rjust(nodeNoColumNum) + 
                "Young".rjust(columNum) + "Poisson".rjust(columNum) + "Density".rjust(columNum) + "\n")
        f.write("-" * columNum * 5 + "-" * nodeNoColumNum + "\n")
        for elem in self.elements:
            strNo = str(elem.no).rjust(columNum)
            strType = str(elem.__class__.__name__ ).rjust(columNum)
            strNodeNo = ""
            for node in elem.nodes:
                strNodeNo += " " + str(node.no)
            strNodeNo = strNodeNo.rjust(nodeNoColumNum)
            strYoung = str(format(elem.young, floatDigits).rjust(columNum))
            strPoisson = "None".rjust(columNum)
            if hasattr(elem, 'poisson'):
                strPoisson = str(format(elem.poisson, floatDigits).rjust(columNum))
            strDensity = "None".rjust(columNum)
            if not elem.density is None:
                strDensity = str(format(elem.density, floatDigits).rjust(columNum))
            f.write(strNo + strType + strNodeNo + strYoung + strPoisson + strDensity + "\n")
        f.write("\n")

        # 単点拘束情報を出力する
        f.write("***** SPC Constraint Data ******\n")
        f.write("NodeNo".rjust(columNum) + "X Displacement".rjust(columNum) + "Y Displacement".rjust(columNum) + "Z Displacement".rjust(columNum) +"\n")
        f.write("-" * columNum * 4 + "\n")
        vecd = self.bound.makeDispVector()
        for i in range(len(self.nodes)):
            flg = False
            for j in range(self.nodeDof):
                if not vecd[self.nodeDof * i + j] == None:
                    flg = True
            if flg == True:
                strNo = str(i + 1).rjust(columNum)
                strXDisp = str(format(vecd[self.nodeDof * i], floatDigits).rjust(columNum))
                strYDisp = str(format(vecd[self.nodeDof * i + 1], floatDigits).rjust(columNum))
                strZDisp = str(format(vecd[self.nodeDof * i + 2], floatDigits).rjust(columNum))
                f.write(strNo + strXDisp + strYDisp + strZDisp + "\n")
        f.write("\n")

        # 荷重条件を出力する(等価節点力も含む)
        f.write("***** Nodal Force Data ******\n")
        f.write("NodeNo".rjust(columNum) + "X Force".rjust(columNum) + "Y Force".rjust(columNum) + "Z Force".rjust(columNum) +"\n")
        f.write("-" * columNum * 4 + "\n")
        vecf = self.makeForceVector()
        for i in range(len(self.nodes)):
            flg = False
            for j in range(self.nodeDof):
                if not vecf[self.nodeDof * i + j] == None:
                    flg = True
            if flg == True:
                strNo = str(i + 1).rjust(columNum)
                strXForce = str(format(vecf[self.nodeDof * i], floatDigits).rjust(columNum))
                strYForce = str(format(vecf[self.nodeDof * i + 1], floatDigits).rjust(columNum))
                strZForce = str(format(vecf[self.nodeDof * i + 2], floatDigits).rjust(columNum))
                f.write(strNo + strXForce + strYForce + strZForce + "\n")
        f.write("\n")

        # 結果データのタイトルを書きこむ
        f.write("**********************************\n")
        f.write("*          Result Data           *\n")
        f.write("**********************************\n")
        f.write("\n")

        # 変位のデータを出力する
        f.write("***** Displacement Data ******\n")
        f.write("NodeNo".rjust(columNum) + "Magnitude".rjust(columNum) + "X Displacement".rjust(columNum) +
                "Y Displacement".rjust(columNum) + "Z Displacement".rjust(columNum) + "\n")
        f.write("-" * columNum * 5 + "\n")
        for i in range(len(self.nodes)):
            strNo = str(i + 1).rjust(columNum)
            mag = np.linalg.norm(np.array((self.vecDisp[self.nodeDof * i], self.vecDisp[self.nodeDof * i + 1], self.vecDisp[self.nodeDof * i + 2])))
            strMag = str(format(mag, floatDigits).rjust(columNum))
            strXDisp = str(format(self.vecDisp[self.nodeDof * i], floatDigits).rjust(columNum))
            strYDisp = str(format(self.vecDisp[self.nodeDof * i + 1], floatDigits).rjust(columNum))
            strZDisp = str(format(self.vecDisp[self.nodeDof * i + 2], floatDigits).rjust(columNum))
            f.write(strNo + strMag + strXDisp + strYDisp + strZDisp + "\n")            
        f.write("\n")
        
        # ひずみのデータを出力する
        f.write("***** Strain Data ******\n")
        f.write("ElementNo".rjust(columNum) + "epsilon_xx".rjust(columNum) +
                "epsilin_yy".rjust(columNum) + "epsilon_zz".rjust(columNum) + "epsilon_xy".rjust(columNum) + 
                "epsilon_yz".rjust(columNum) + "epsilon_zx".rjust(columNum) + "\n")
        f.write("-" * columNum * 7 + "\n")
        for i in range(len(self.elements)):
            strNo = str(i + 1).rjust(columNum)
            strXXStrain = str(format(self.vecStrain[self.nodeDof * 2 * i    ], floatDigits).rjust(columNum))
            strYYStrain = str(format(self.vecStrain[self.nodeDof * 2 * i + 1], floatDigits).rjust(columNum))
            strZZStrain = str(format(self.vecStrain[self.nodeDof * 2 * i + 2], floatDigits).rjust(columNum))
            strXYStrain = str(format(self.vecStrain[self.nodeDof * 2 * i + 3], floatDigits).rjust(columNum))
            strYZStrain = str(format(self.vecStrain[self.nodeDof * 2 * i + 4], floatDigits).rjust(columNum))
            strZXStrain = str(format(self.vecStrain[self.nodeDof * 2 * i + 5], floatDigits).rjust(columNum))
        f.write("\n")
        
        # 応力のデータを出力する
        f.write("***** Stress Data ******\n")
        f.write("ElementNo".rjust(columNum) + "sigma_xx".rjust(columNum) +
                "sigma_yy".rjust(columNum) + "sigma_zz".rjust(columNum) + "sigma_xy".rjust(columNum) + 
                "sigma_yz".rjust(columNum) + "sigma_zx".rjust(columNum) + "\n")
        f.write("-" * columNum * 7 + "\n")
        for i in range(len(self.elements)):
            strNo = str(i + 1).rjust(columNum)
            strXXStrain = str(format(self.vecStress[self.nodeDof * 2 * i    ], floatDigits).rjust(columNum))
            strYYStrain = str(format(self.vecStress[self.nodeDof * 2 * i + 1], floatDigits).rjust(columNum))
            strZZStrain = str(format(self.vecStress[self.nodeDof * 2 * i + 2], floatDigits).rjust(columNum))
            strXYStrain = str(format(self.vecStress[self.nodeDof * 2 * i + 3], floatDigits).rjust(columNum))
            strYZStrain = str(format(self.vecStress[self.nodeDof * 2 * i + 4], floatDigits).rjust(columNum))
            strZXStrain = str(format(self.vecStress[self.nodeDof * 2 * i + 5], floatDigits).rjust(columNum))
        f.write("\n")
         
        for i in range(len(self.nodes)):
            strNo = str(i + 1).rjust(columNum)
            mag = np.linalg.norm(np.array((self.vecDisp[self.nodeDof * i], self.vecDisp[self.nodeDof * i + 1], self.vecDisp[self.nodeDof * i + 2])))
            strMag = str(format(mag, floatDigits).rjust(columNum))
            strXDisp = str(format(self.vecDisp[self.nodeDof * i], floatDigits).rjust(columNum))
            strYDisp = str(format(self.vecDisp[self.nodeDof * i + 1], floatDigits).rjust(columNum))
            strZDisp = str(format(self.vecDisp[self.nodeDof * i + 2], floatDigits).rjust(columNum))
            f.write(strNo + strMag + strXDisp + strYDisp + strZDisp + "\n")            
        f.write("\n")

        # 反力のデータを出力する
        f.write("***** Reaction Force Data ******\n")
        f.write("NodeNo".rjust(columNum) + "Magnitude".rjust(columNum) + "X Force".rjust(columNum) + "Y Force".rjust(columNum) + "Z Force".rjust(columNum) + "\n")
        f.write("-" * columNum * 5 + "\n")
        for i in range(len(self.nodes)):
            strNo = str(i + 1).rjust(columNum)
            mag = np.linalg.norm(np.array((self.vecRF[self.nodeDof * i], self.vecRF[self.nodeDof * i + 1], self.vecRF[self.nodeDof * i + 2])))
            strMag = str(format(mag, floatDigits).rjust(columNum))
            strXForce = str(format(self.vecRF[self.nodeDof * i], floatDigits).rjust(columNum))
            strYForce = str(format(self.vecRF[self.nodeDof * i + 1], floatDigits).rjust(columNum))
            strZForce = str(format(self.vecRF[self.nodeDof * i + 2], floatDigits).rjust(columNum))
            f.write(strNo + strMag + strXForce + strYForce + strZForce + "\n")            
        f.write("\n")

        # ファイルを閉じる
        f.close()

    def outputVtk(self, filePath1, filePath2):
        # ファイルを作成し、開く
        
        ############################
        #         File 1           #
        ############################
        
        f = open(filePath1 + ".vtk", 'w')
        floatDigits = ".10g"
        
        # おまじない
        f.write("# vtk DataFile Version 2.0\n")
        f.write("Displacement.\n")
        f.write("ASCII\n")
        f.write("DATASET UNSTRUCTURED_GRID\n")
        f.write("\n")

        # 節点情報
        f.write("POINTS {} float\n".format(len(self.nodes)))
        for node in self.nodes:
          f.write("{0} {1} {2}\n".format(node.x, node.y, node.z))
        f.write("\n")

        # 要素情報
        f.write("CELLS {0} {1}\n".format(len(self.elements), len(self.elements)*5))
        for elem in self.elements:
          strNodeNo = ""
          for node in elem.nodes:
                strNodeNo += " " + str(node.no)
          f.write("4 {}\n".format(strNodeNo))
        f.write("CELL_TYPES {}\n".format(len(self.elements)))
        for i in range(len(self.elements)):
          f.write("10\n")
        f.write("\n")

        # 変位情報
        f.write("POINT_DATA {}\n".format(len(self.nodes)))
        f.write("SCALARS Dispracement_Norm float\n")
        f.write("LOOKUP_TABLE default\n")

        #　ノルム
        for i in range(len(self.nodes)):
          mag = np.linalg.norm(np.array((self.vecDisp[self.nodeDof * i], self.vecDisp[self.nodeDof * i + 1], self.vecDisp[self.nodeDof * i + 2])))
          f.write("{0}\n".format(mag))
        f.write("\n")

        #　x方向変位
        #for i in range(len(self.nodes)):
        #  f.write("{0}\n".format(self.vecDisp[self.nodeDof * i]))

        # 反力情報
        f.write("SCALARS Counterforce_helmet float\n")
        f.write("LOOKUP_TABLE default\n")
        for i in range(len(self.nodes)):
          mag = np.linalg.norm(np.array((self.vecRF[self.nodeDof * i], self.vecRF[self.nodeDof * i + 1], self.vecRF[self.nodeDof * i + 2])))
          f.write("{0}\n".format(mag))
        f.write("\n")        

        # ひずみ情報
        f.write("CELL_DATA {}\n".format(len(self.elements)))
        f.write("SCALARS Strain_xx float\n")
        f.write("LOOKUP_TABLE default\n")

        for i in range(len(self.elements)):
            f.write("{0}\n".format(self.vecStrain[self.nodeDof * 2 * i]))
        f.write("\n")

        #f.write("CELL_DATA {}\n".format(len(self.elements)))
        f.write("SCALARS Strain_yy float\n")
        f.write("LOOKUP_TABLE default\n")

        for i in range(len(self.elements)):
            f.write("{0}\n".format(self.vecStrain[self.nodeDof * 2 * i + 1]))
        f.write("\n")

        #f.write("CELL_DATA {}\n".format(len(self.elements)))
        f.write("SCALARS Strain_zz float\n")
        f.write("LOOKUP_TABLE default\n")

        for i in range(len(self.elements)):
            f.write("{0}\n".format(self.vecStrain[self.nodeDof * 2 * i + 2]))
        f.write("\n")

        #f.write("CELL_DATA {}\n".format(len(self.elements)))
        f.write("SCALARS Strain_xy float\n")
        f.write("LOOKUP_TABLE default\n")

        for i in range(len(self.elements)):
            f.write("{0}\n".format(self.vecStrain[self.nodeDof * 2 * i + 3]))
        f.write("\n")

        #f.write("CELL_DATA {}\n".format(len(self.elements)))
        f.write("SCALARS Strain_yz float\n")
        f.write("LOOKUP_TABLE default\n")

        for i in range(len(self.elements)):
            f.write("{0}\n".format(self.vecStrain[self.nodeDof * 2 * i + 4]))
        f.write("\n")

        #f.write("CELL_DATA {}\n".format(len(self.elements)))
        f.write("SCALARS Strain_zx float\n")
        f.write("LOOKUP_TABLE default\n")

        for i in range(len(self.elements)):
            f.write("{0}\n".format(self.vecStrain[self.nodeDof * 2 * i + 5]))
        f.write("\n")
        
        f.close()
        
        ############################
        #         File 2           #
        ############################
        
        # For visualization head's information
        
        f2 = open(filePath2 + ".vtk", 'w')
        floatDigits = ".10g"
        
        # おまじない
        f2.write("# vtk DataFile Version 2.0\n")
        f2.write("Displacement.\n")
        f2.write("ASCII\n")
        f2.write("DATASET UNSTRUCTURED_GRID\n")
        f2.write("\n")
        
        # 節点情報
        f2.write("POINTS {} float\n".format(len(self.nodes_head)))
        for node in self.nodes_head:
          f2.write("{0} {1} {2}\n".format(node.x, node.y, node.z))
        f2.write("\n")

        # 要素情報
        f2.write("CELLS {0} {1}\n".format(len(self.elements_head), len(self.elements_head)*5))
        for elem in self.elements_head:
          strNodeNo = ""
          for node in elem.nodes:
                strNodeNo += " " + str(node.no)
          f2.write("4 {}\n".format(strNodeNo))
        f2.write("CELL_TYPES {}\n".format(len(self.elements_head)))
        for i in range(len(self.elements_head)):
          f2.write("10\n")
        f2.write("\n")
        
        # 反力のノルム
        f2.write("CELL_DATA {}\n".format(len(self.elements_head)))
        f2.write("SCALARS Counterforce_head_norm float\n")
        f2.write("LOOKUP_TABLE default\n")
        
        # bc_element_head: ヘルメットと接触する面を含む四面体
        # bc_RF_helmet: 頭と接触する節点
        
        for i in range(len(self.elements_head)):
            # 接触面
            if i in bc_element_head:
                ind = bc_element_head.index(i)
                mag = np.linalg.norm(np.array((self.vecRF[self.nodeDof * ind], self.vecRF[self.nodeDof * ind + 1], self.vecRF[self.nodeDof * ind + 2])))
                f2.write("{0}\n".format(mag))
            
            # 非接触面
            else:
                f2.write("{0}\n".format(0))
        f2.write("\n")
        
        # 反力のうち頭に内向き垂直な方向のノルム
        f2.write("SCALARS Counterforce_head_normal float\n")
        f2.write("LOOKUP_TABLE default\n")
        
        for i in range(len(self.elements_head)):
            # 接触面
            if i in bc_element_head:
                ind = bc_element_head.index(i)
                f = bc_face_head[ind]
                
                n0 = f_node[3*f  ]
                n1 = f_node[3*f+1]
                n2 = f_node[3*f+2]
                
                v0 = x_head[n2] - x_head[n0] 
                v1 = x_head[n1] - x_head[n0]
                
                n = np.cross(v0, v1)/np.linalg.norm(np.cross(v0, v1))
                vec_tmp = [self.vecRF[self.nodeDof * ind], self.vecRF[self.nodeDof * ind + 1], self.vecRF[self.nodeDof * ind + 2]]
                
                mag = np.dot(n, np.array(vec_tmp))
                if mag < 0:
                    mag = 0.
                f2.write("{0}\n".format(mag))
            
            # 非接触面
            else:
                f2.write("{0}\n".format(0))
        f2.write("\n")
        
        f2.close()


### Import mesh data

In [ ]:
!conda install -c conda-forge meshio -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/satoshi_noguchi/opt/anaconda3/envs/test_env

  added / updated specs:
    - meshio


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    c-ares-1.18.1              |       h0d85af4_0         100 KB  conda-forge
    ca-certificates-2022.12.7  |       h033912b_0         142 KB  conda-forge
    cached-property-1.5.2      |       hd8ed1ab_1           4 KB  conda-forge
    cached_property-1.5.2      |     pyha770c72_1          11 KB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    cftime-1.6.2               |  py310h936d966_1         218 KB  conda-forge
    curl-7.88.1                |       h6c4

In [ ]:
import meshio

In [ ]:
#head_mesh = meshio.read("/Users/ngtsts/Desktop/Research/副業/Berry/Data/mesh/sample2/MSH/SNO8248FR_scan_gmsh_uniform_5.msh")
head_mesh = meshio.read("/Users/satoshi_noguchi/Desktop/Research/副業/Berry/Data/mesh/sample2/MSH/SNO8248FR_scan_gmsh_uniform_5.msh")
x_head = head_mesh.points
ele_head = head_mesh.cells_dict["tetra"]

In [ ]:
#helmet_mesh = meshio.read("/Users/ngtsts/Desktop/Research/副業/Berry/Data/mesh/sample2/MSH/SNO8248FR_helmet_gmsh_uniform_5.msh")
helmet_mesh = meshio.read("/Users/satoshi_noguchi/Desktop/Research/副業/Berry/Data/mesh/sample2/MSH/SNO8248FR_helmet_gmsh_uniform_5.msh")
x = helmet_mesh.points
ele = helmet_mesh.cells_dict["tetra"]

In [ ]:
head_mesh

<meshio mesh object>
  Number of points: 11621
  Number of cells:
    triangle: 7174
    tetra: 61007
  Cell data: gmsh:physical, gmsh:geometrical
  Field data: surface1, surface2, surface3, volume1

In [ ]:
helmet_mesh

<meshio mesh object>
  Number of points: 4556
  Number of cells:
    triangle: 9108
    tetra: 13281
  Cell data: gmsh:physical, gmsh:geometrical
  Field data: surface1, surface2, surface3, surface4, surface5, surface6, surface7, surface8, surface9, volume1

### Identify contact points


In [ ]:
def tetra_volume_head(n0, n1, n2, n3):
  v10 = x_head[n1] - x_head[n0]
  v20 = x_head[n2] - x_head[n0]
  v30 = x_head[n3] - x_head[n0]

  V = np.dot(np.cross(v10, v20), v30)/6

  if V < 0:
    print("右手系になってる？")
  else:
    return V

def tetra_volume_helmet(n0, n1, n2, n3):
  v10 = x[n1] - x[n0]
  v20 = x[n2] - x[n0]
  v30 = x[n3] - x[n0]

  V = np.dot(np.cross(v10, v20), v30)/6

  if V < 0:
    print("右手系になってる？")
  else:
    return V

In [ ]:
V_max = -1
V = 0
V_total = 0

for i in range(len(ele_head)):
  n0, n1, n2, n3 = ele_head[i]
  V = tetra_volume_head(n0, n1, n2, n3)
  V_total+=V
  if V > V_max:
    V_max = V

av_len_head = (V_total/len(ele_head))** (1/3)
print("The biggest element in head: {:3f}".format(V_max))
print("The average length in head: {:3f}".format(av_len_head))

V_max = -1
V = 0
V_total = 0

for i in range(len(ele)):
  n0, n1, n2, n3 = ele[i]
  V = tetra_volume_helmet(n0, n1, n2, n3)
  V_total+=V
  if V > V_max:
    V_max = V

av_len_helment = (V_total/len(ele))** (1/3)

print("The biggest element in helmet: {:3f}".format(V_max))
print("The average length in helment: {:3f}".format(av_len_helment))

The biggest element in head: 81.489318
The average length in head: 3.028684
The biggest element in helmet: 25.392355
The average length in helment: 2.363473


#### Identify geometry of head


In [ ]:
n_eh = len(ele_head)
n_ph = len(x_head)

e_link_start = np.zeros(n_ph+1, dtype=np.int32)

for i in range(n_eh):
  n0, n1, n2, n3 = ele_head[i]
  e_link_start[n0] += 1
  e_link_start[n1] += 1
  e_link_start[n2] += 1
  e_link_start[n3] += 1

for i in range(n_ph): e_link_start[i] += e_link_start[i-1]
for i in reversed(range(1, n_ph+1)): e_link_start[i] = e_link_start[i-1]
e_link_start[0] = 0

In [ ]:
e_link = np.full(e_link_start[n_ph], -1, dtype=np.int32)

for i in range(n_eh):
  for n in ele_head[i]:
    jstart = e_link_start[n]
    jend = e_link_start[n+1]
    for j in range(jstart,jend):
      if e_link[j] == -1:
        e_link[j] = i
        #j = jend
        break

print("e_link done.")

e_link done.


In [ ]:
n_link_start = np.zeros(n_ph+1, dtype=np.int32)
n_link_list = []

for i in range(n_ph):
  n_link_tmp = []
  jstart = e_link_start[i]
  jend = e_link_start[i+1]
  for j in range(jstart,jend):
    el = e_link[j]
    n0, n1, n2, n3 = ele_head[el]
    n_link_tmp += [n0, n1, n2, n3]

  n_link_tmp.sort()
  n_link_tmp = list(dict.fromkeys(n_link_tmp))
  n_link_tmp.remove(i)
  n_link_list += n_link_tmp
  
  n_link_start[i+1] = n_link_start[i] + len(n_link_tmp)

n_link = np.array(n_link_list, dtype=np.int32)

print("n_link done.")

n_link done.


In [ ]:
def Comb(n00: int, n11: int, n22: int, n2: int, n1: int, n0: int):
  if n00==n2 and n11==n1 and n22==n0:
    return True
  elif n00==n1 and n11==n0 and n22==n2:
    return True
  elif n00==n0 and n11==n2 and n22==n1:
    return True
  else:
    return False

In [ ]:
e_f_link = np.full(4*n_eh, -1, dtype=np.int32)
f_node = []
f_counter = 0

for i in range(n_eh):
  itmp = 4*i
  n0, n1, n2, n3 = ele_head[i]

  if e_f_link[itmp] == -1:
    e_f_link[itmp] = f_counter
    for j in range(e_link_start[n0], e_link_start[n0+1]):
      jtmp = 4*e_link[j]
      n00, n11, n22, n33 = ele_head[e_link[j]]
      if Comb(n00, n11, n22, n2, n1, n0) and e_f_link[jtmp]==-1:
        e_f_link[jtmp]=f_counter
        break
      elif Comb(n33, n11, n00, n2, n1, n0) and e_f_link[jtmp+1]==-1:
        e_f_link[jtmp+1]=f_counter
        break
      elif Comb(n00, n22, n33, n2, n1, n0) and e_f_link[jtmp+2]==-1:
        e_f_link[jtmp+2]=f_counter
        break
      elif Comb(n33, n22, n11, n2, n1, n0) and e_f_link[jtmp+3]==-1:
        e_f_link[jtmp+3]=f_counter
        break
    f_counter=f_counter+1
    f_node.append(n0)
    f_node.append(n2)
    f_node.append(n1)

  if e_f_link[itmp+1] == -1:
    e_f_link[itmp+1] = f_counter
    for j in range(e_link_start[n0], e_link_start[n0+1]):
      jtmp = 4*e_link[j]
      n00, n11, n22, n33 = ele_head[e_link[j]]
      if Comb(n00, n11, n22, n0, n1, n3) and e_f_link[jtmp]==-1:
        e_f_link[jtmp]=f_counter
        break
      elif Comb(n33, n11, n00, n0, n1, n3) and e_f_link[jtmp+1]==-1:
        e_f_link[jtmp+1]=f_counter
        break
      elif Comb(n00, n22, n33, n0, n1, n3) and e_f_link[jtmp+2]==-1:
        e_f_link[jtmp+2]=f_counter
        break
      elif Comb(n33, n22, n11, n0, n1, n3) and e_f_link[jtmp+3]==-1:
        e_f_link[jtmp+3]=f_counter
        break
    f_counter=f_counter+1
    f_node.append(n0)
    f_node.append(n1)
    f_node.append(n3)

  if e_f_link[itmp+2] == -1:
    e_f_link[itmp+2] = f_counter
    for j in range(e_link_start[n0], e_link_start[n0+1]):
      jtmp = 4*e_link[j]
      n00, n11, n22, n33 = ele_head[e_link[j]]
      if Comb(n00, n11, n22, n3, n2, n0) and e_f_link[jtmp]==-1:
        e_f_link[jtmp]=f_counter
        break
      elif Comb(n33, n11, n00, n3, n2, n0) and e_f_link[jtmp+1]==-1:
        e_f_link[jtmp+1]=f_counter
        break
      elif Comb(n00, n22, n33, n3, n2, n0) and e_f_link[jtmp+2]==-1:
        e_f_link[jtmp+2]=f_counter
        break
      elif Comb(n33, n22, n11, n3, n2, n0) and e_f_link[jtmp+3]==-1:
        e_f_link[jtmp+3]=f_counter
        break
    f_counter=f_counter+1
    f_node.append(n3)
    f_node.append(n2)
    f_node.append(n0)

  if e_f_link[itmp+3] == -1:
    e_f_link[itmp+3] = f_counter
    for j in range(e_link_start[n1], e_link_start[n1+1]):
      jtmp = 4*e_link[j]
      n00, n11, n22, n33 = ele_head[e_link[j]]
      if Comb(n00, n11, n22, n1, n2, n3) and e_f_link[jtmp]==-1:
        e_f_link[jtmp]=f_counter
        break
      elif Comb(n33, n11, n00, n1, n2, n3) and e_f_link[jtmp+1]==-1:
        e_f_link[jtmp+1]=f_counter
        break
      elif Comb(n00, n22, n33, n1, n2, n3) and e_f_link[jtmp+2]==-1:
        e_f_link[jtmp+2]=f_counter
        break
      elif Comb(n33, n22, n11, n1, n2, n3) and e_f_link[jtmp+3]==-1:
        e_f_link[jtmp+3]=f_counter
        break
    f_counter=f_counter+1
    f_node.append(n1)
    f_node.append(n2)
    f_node.append(n3)

In [ ]:
f_node = np.array(f_node, dtype=np.int32)

In [ ]:
n_face = f_counter
print("面の数は{}.".format(n_face))

面の数は125601.


In [ ]:
len(e_f_link)

244028

In [ ]:
f_e_link = np.full(2*n_face, -1, dtype=np.int32)

for i in range(n_eh):
  itmp = 4*i
  f0 = e_f_link[itmp  ]
  f1 = e_f_link[itmp+1]
  f2 = e_f_link[itmp+2]
  f3 = e_f_link[itmp+3]
  faces = [f0, f1, f2, f3]

  for f in faces:
    if f_e_link[2*f]==-1:
      f_e_link[2*f]=i
    elif f_e_link[2*f+1]==-1:
      f_e_link[2*f+1]=i
    else:
      print("Something wrong!")

In [ ]:
f_e_link

array([    0,   602,     0, ..., 61004, 61005, 61006], dtype=int32)

In [ ]:
#境界を構成する面を取り出す
bf_list = []

for f in range(n_face):
  if f_e_link[2*f+1]==-1:
    bf_list.append(f)

print("境界を構成する面の数は{}.".format(len(bf_list)))

境界を構成する面の数は7174.


#### Identify Geometry of helment

In [ ]:
n_e = len(ele)
n_p = len(x)

e_link_start_hel = np.zeros(n_p+1, dtype=np.int32)

for i in range(n_e):
  n0, n1, n2, n3 = ele[i]
  e_link_start_hel[n0] += 1
  e_link_start_hel[n1] += 1
  e_link_start_hel[n2] += 1
  e_link_start_hel[n3] += 1

for i in range(n_p): e_link_start_hel[i] += e_link_start_hel[i-1]
for i in reversed(range(1, n_p+1)): e_link_start_hel[i] = e_link_start_hel[i-1]
e_link_start_hel[0] = 0

In [ ]:
e_link_hel = np.full(e_link_start_hel[n_p], -1, dtype=np.int32)

for i in range(n_e):
  for n in ele[i]:
    jstart = e_link_start_hel[n]
    jend = e_link_start_hel[n+1]
    for j in range(jstart,jend):
      if e_link_hel[j] == -1:
        e_link_hel[j] = i
        #j = jend
        break

print("e_link done.")

e_link done.


In [ ]:
n_link_start_hel = np.zeros(n_p+1, dtype=np.int32)
n_link_list_hel = []

for i in range(n_p):
  n_link_tmp = []
  jstart = e_link_start_hel[i]
  jend = e_link_start_hel[i+1]
  for j in range(jstart,jend):
    el = e_link_hel[j]
    n0, n1, n2, n3 = ele[el]
    n_link_tmp += [n0, n1, n2, n3]

  n_link_tmp.sort()
  n_link_tmp = list(dict.fromkeys(n_link_tmp))
  n_link_tmp.remove(i)
  n_link_list_hel += n_link_tmp
  
  n_link_start_hel[i+1] = n_link_start_hel[i] + len(n_link_tmp)

n_link_hel = np.array(n_link_list_hel, dtype=np.int32)

print("n_link done.")

n_link done.


In [ ]:
e_f_link_hel = np.full(4*n_e, -1, dtype=np.int32)
f_node_hel = []
f_counter = 0

for i in range(n_e):
  itmp = 4*i
  n0, n1, n2, n3 = ele[i]

  if e_f_link_hel[itmp] == -1:
    e_f_link_hel[itmp] = f_counter
    for j in range(e_link_start_hel[n0], e_link_start_hel[n0+1]):
      jtmp = 4*e_link_hel[j]
      n00, n11, n22, n33 = ele[e_link_hel[j]]
      if Comb(n00, n11, n22, n2, n1, n0) and e_f_link_hel[jtmp]==-1:
        e_f_link_hel[jtmp]=f_counter
        break
      elif Comb(n33, n11, n00, n2, n1, n0) and e_f_link_hel[jtmp+1]==-1:
        e_f_link_hel[jtmp+1]=f_counter
        break
      elif Comb(n00, n22, n33, n2, n1, n0) and e_f_link_hel[jtmp+2]==-1:
        e_f_link_hel[jtmp+2]=f_counter
        break
      elif Comb(n33, n22, n11, n2, n1, n0) and e_f_link_hel[jtmp+3]==-1:
        e_f_link_hel[jtmp+3]=f_counter
        break
    f_counter=f_counter+1
    f_node_hel.append(n0)
    f_node_hel.append(n2)
    f_node_hel.append(n1)

  if e_f_link_hel[itmp+1] == -1:
    e_f_link_hel[itmp+1] = f_counter
    for j in range(e_link_start_hel[n0], e_link_start_hel[n0+1]):
      jtmp = 4*e_link_hel[j]
      n00, n11, n22, n33 = ele[e_link_hel[j]]
      if Comb(n00, n11, n22, n0, n1, n3) and e_f_link_hel[jtmp]==-1:
        e_f_link_hel[jtmp]=f_counter
        break
      elif Comb(n33, n11, n00, n0, n1, n3) and e_f_link_hel[jtmp+1]==-1:
        e_f_link_hel[jtmp+1]=f_counter
        break
      elif Comb(n00, n22, n33, n0, n1, n3) and e_f_link_hel[jtmp+2]==-1:
        e_f_link_hel[jtmp+2]=f_counter
        break
      elif Comb(n33, n22, n11, n0, n1, n3) and e_f_link_hel[jtmp+3]==-1:
        e_f_link_hel[jtmp+3]=f_counter
        break
    f_counter=f_counter+1
    f_node_hel.append(n0)
    f_node_hel.append(n1)
    f_node_hel.append(n3)

  if e_f_link_hel[itmp+2] == -1:
    e_f_link_hel[itmp+2] = f_counter
    for j in range(e_link_start_hel[n0], e_link_start_hel[n0+1]):
      jtmp = 4*e_link_hel[j]
      n00, n11, n22, n33 = ele[e_link_hel[j]]
      if Comb(n00, n11, n22, n3, n2, n0) and e_f_link_hel[jtmp]==-1:
        e_f_link_hel[jtmp]=f_counter
        break
      elif Comb(n33, n11, n00, n3, n2, n0) and e_f_link_hel[jtmp+1]==-1:
        e_f_link_hel[jtmp+1]=f_counter
        break
      elif Comb(n00, n22, n33, n3, n2, n0) and e_f_link_hel[jtmp+2]==-1:
        e_f_link_hel[jtmp+2]=f_counter
        break
      elif Comb(n33, n22, n11, n3, n2, n0) and e_f_link_hel[jtmp+3]==-1:
        e_f_link_hel[jtmp+3]=f_counter
        break
    f_counter=f_counter+1
    f_node_hel.append(n3)
    f_node_hel.append(n2)
    f_node_hel.append(n0)

  if e_f_link_hel[itmp+3] == -1:
    e_f_link_hel[itmp+3] = f_counter
    for j in range(e_link_start_hel[n1], e_link_start_hel[n1+1]):
      jtmp = 4*e_link_hel[j]
      n00, n11, n22, n33 = ele[e_link_hel[j]]
      if Comb(n00, n11, n22, n1, n2, n3) and e_f_link_hel[jtmp]==-1:
        e_f_link_hel[jtmp]=f_counter
        break
      elif Comb(n33, n11, n00, n1, n2, n3) and e_f_link_hel[jtmp+1]==-1:
        e_f_link_hel[jtmp+1]=f_counter
        break
      elif Comb(n00, n22, n33, n1, n2, n3) and e_f_link_hel[jtmp+2]==-1:
        e_f_link_hel[jtmp+2]=f_counter
        break
      elif Comb(n33, n22, n11, n1, n2, n3) and e_f_link_hel[jtmp+3]==-1:
        e_f_link_hel[jtmp+3]=f_counter
        break
    f_counter=f_counter+1
    f_node_hel.append(n1)
    f_node_hel.append(n2)
    f_node_hel.append(n3)

In [ ]:
f_node_hel = np.array(f_node_hel, dtype=np.int32)

In [ ]:
n_face_hel = f_counter
print("面の数は{}.".format(n_face))

面の数は125601.


In [ ]:
len(e_f_link_hel)

53124

In [ ]:
f_e_link_hel = np.full(2*n_face_hel, -1, dtype=np.int32)

for i in range(n_e):
  itmp = 4*i
  f0 = e_f_link_hel[itmp  ]
  f1 = e_f_link_hel[itmp+1]
  f2 = e_f_link_hel[itmp+2]
  f3 = e_f_link_hel[itmp+3]
  faces = [f0, f1, f2, f3]

  for f in faces:
    if f_e_link_hel[2*f]==-1:
      f_e_link_hel[2*f]=i
    elif f_e_link_hel[2*f+1]==-1:
      f_e_link_hel[2*f+1]=i
    else:
      print("Something wrong!")

In [ ]:
#境界を構成する面を取り出す
bf_list_hel = []

for f in range(n_face_hel):
  if f_e_link_hel[2*f+1]==-1:
    bf_list_hel.append(f)

print("境界を構成する麺の数は{}.".format(len(bf_list_hel)))

境界を構成する麺の数は9108.


In [ ]:
bc_node_tmp = []

for f in bf_list_hel:
  bc_node_tmp.append(f_node_hel[3*f  ])
  bc_node_tmp.append(f_node_hel[3*f+1])
  bc_node_tmp.append(f_node_hel[3*f+2])

bc_node_tmp.sort()
bc_node = list(dict.fromkeys(bc_node_tmp))

print(len(bc_node))
print(len(x))

4556
4556


#### Create bc_list

In [ ]:
def det(a, b, c):
  M = np.stack([a, b, c])
  return np.linalg.det(M)

In [ ]:
def contact(f, p_ind):
  # a*u + b*v + c*t = d
  a = x_head[f_node[3*f+1]] - x_head[f_node[3*f  ]]
  b = x_head[f_node[3*f+2]] - x_head[f_node[3*f  ]]
  c = - np.array([0, 0, 1]) # ray
  d = x[p_ind] - x_head[f_node[3*f  ]]

  u = det(d, b, c)/det(a, b, c)
  v = det(a, d, c)/det(a, b, c)
  t = det(a, b, d)/det(a, b, c)

  if u < 0 or 1 < u:
    return False
  elif v < 0 or 1 < (u+v):
    return False
  elif t < 0:
    return False
  else:
    return True

In [ ]:
from tqdm.notebook import tqdm
bc_list = []

for ind in tqdm(range(len(bc_node))):
  counter = 0
  for f in bf_list:
    v = x_head[f_node[3*f  ]] - x[ind]
    u = x_head[f_node[3*f+1]] - x[ind]
    w = x_head[f_node[3*f+2]] - x[ind]
    
    v[2] = 0
    u[2] = 0
    w[2] = 0

    if np.linalg.norm(v,ord=1) < av_len_head*7:
      if np.linalg.norm(u,ord=1)  < av_len_head*7:
        if np.linalg.norm(w,ord=1)  < av_len_head*7:
          if contact(f, ind):
            #print("FIND!")
            counter+=1

  if counter%2==1:
    #print("CONTACT!")
    bc_list.append(ind)

  0%|          | 0/4556 [00:00<?, ?it/s]

In [ ]:
len(bc_list)

71

In [ ]:
import csv
import os 
path = './bc_list.csv'
#if not os.path.exists(path):
#  os.mkdir(path)

with open(path, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["x", "y", "z", "1"])
    for ind in range(len(bc_list)):
      writer.writerow([x[bc_list[ind], 0], x[bc_list[ind], 1], x[bc_list[ind], 2], 1])

### Boundary conditions

In [ ]:
bf_barycenter_head = []

for f in bf_list:
  n0 = f_node[3*f  ]
  n1 = f_node[3*f+1]
  n2 = f_node[3*f+2]

  x_bar = (x_head[n0] + x_head[n1] + x_head[n2])/3
  bf_barycenter_head.append(x_bar)

print(len(bf_barycenter_head))

7174


In [ ]:
def distance(x1, y1, z1, x2, y2, z2):
  distance = np.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
  return distance

def min_distance_bar(n_ind):
  x1 = x[n_ind, 0]
  y1 = x[n_ind, 1]
  z1 = x[n_ind, 2]

  for i, bar in enumerate(bf_barycenter_head):
    dis_tmp = distance(x1, y1, z1, bar[0], bar[1], bar[2])
    if i == 0:
      min_distance = dis_tmp
      min_f = bf_list[i]
    elif dis_tmp < min_distance:
      min_distance = dis_tmp
      min_f = bf_list[i]
  
  return min_f, min_distance

In [ ]:
def perpendicular(f, p_ind):
    # f : some face in head's surface
    # n : some node in helmet's surface
    a = x_head[f_node[3*f+1]] - x_head[f_node[3*f  ]]
    b = x_head[f_node[3*f+2]] - x_head[f_node[3*f  ]]
    
    n = np.cross(a, b)
    c = - n/np.linalg.norm(n)
    d = x[p_ind] - x_head[f_node[3*f  ]]

    u = det(d, b, c)/det(a, b, c)
    v = det(a, d, c)/det(a, b, c)
    t = det(a, b, d)/det(a, b, c)
    
    q = x_head[f_node[3*f  ]] + u*a + v*b
    
    return q

In [ ]:
bc_coordinates = []
bc_face_head = []
bc_element_head = []
bc_RF_helmet = []

# bc_face_head: ヘルメットと接触する面
# bc_element_head: ヘルメットと接触する面を含む四面体
# bc_RF_helmet: 頭と接触する節点
# 上の二つの配列の順番は対応する

for n in tqdm(bc_list):
  f, _ = min_distance_bar(n)
  bc_face_head.append(f)
  bc_element_head.append(f_e_link[2*f])
  bc_RF_helmet.append(n)
  if not f_e_link[2*f+1] == -1:
    print("!頭の表面ではない面が入力されている!")
  
  # 垂線の足パターン
  q = perpendicular(f, n)
  bc_coordinates.append(q)

  # 重心パターン あんまり良くない？
  #bc_coordinates.append(bf_barycenter_head[f])

  0%|          | 0/71 [00:00<?, ?it/s]

In [ ]:
# Boundary Condition for force
z_max = - 100
for ind in range(len(x)):
    z_tmp = x[ind, 2]
    if z_tmp > z_max:
        z_max = z_tmp
        max_ind = ind

print(z_max)
print(max_ind)

74.52728220613949
344


### Analysis

In [ ]:
# 材料情報を定義する
young = 1.0e9
poisson = 0.3
density = 950.0
#vecGrav = np.array([0.0, 0.0, -9.81])
nodes = []
elems = []

nodes_head = []
elems_head = []

# ヘルメット
# 節点を定義する
for ind in range(len(x)):
  node = Node(ind, x[ind, 0], x[ind, 1], x[ind, 2])
  nodes.append(node)

# 要素を定義する
for ind in range(len(ele)):
  node0 = Node(ele[ind][0], x[ele[ind][0], 0], x[ele[ind][0], 1], x[ele[ind][0], 2])
  node1 = Node(ele[ind][1], x[ele[ind][1], 0], x[ele[ind][1], 1], x[ele[ind][1], 2])
  node2 = Node(ele[ind][2], x[ele[ind][2], 0], x[ele[ind][2], 1], x[ele[ind][2], 2])
  node3 = Node(ele[ind][3], x[ele[ind][3], 0], x[ele[ind][3], 1], x[ele[ind][3], 2])
  elems.append(C3D4(ind, [node0, node1, node2, node3], young, poisson, density))

# 頭
# 節点を定義する
for ind in range(len(x_head)):
  node = Node(ind, x_head[ind, 0], x_head[ind, 1], x_head[ind, 2])
  nodes_head.append(node)

# 要素を定義する
for ind in range(len(ele_head)):
  node0 = Node(ele_head[ind][0], x_head[ele_head[ind][0], 0], x_head[ele_head[ind][0], 1], x_head[ele_head[ind][0], 2])
  node1 = Node(ele_head[ind][1], x_head[ele_head[ind][1], 0], x_head[ele_head[ind][1], 1], x_head[ele_head[ind][1], 2])
  node2 = Node(ele_head[ind][2], x_head[ele_head[ind][2], 0], x_head[ele_head[ind][2], 1], x_head[ele_head[ind][2], 2])
  node3 = Node(ele_head[ind][3], x_head[ele_head[ind][3], 0], x_head[ele_head[ind][3], 1], x_head[ele_head[ind][3], 2])
  elems_head.append(C3D4(ind, [node0, node1, node2, node3], young, poisson, density))

bound = Boundary(len(nodes))
for i, n_ind in enumerate(bc_list):
    
  bound.addSPC(n_ind, bc_coordinates[i][0] - x[n_ind, 0], bc_coordinates[i][1] - x[n_ind, 1], 
               bc_coordinates[i][2] - x[n_ind, 2])

# force　boundary condition

# 解析を行う
fem = FEM(nodes, elems, nodes_head, elems_head, bound)
fem.analysis()

(array([-0.00211499, -0.01132257,  0.00123677, ...,  0.01047796,
        -0.01187999,  0.01710019]),
 array([ 0.00000000e+00,  2.23517418e-08, -6.51925802e-09, ...,
        -2.08616257e-07,  1.49011612e-08, -3.27825546e-07]))

In [ ]:
fem.outputTxt("test")

In [ ]:
#　出力
fem.outputVtk("test_helmet", "test_head")

In [ ]:
### 確認用

In [ ]:
node0 = Node(0, 0, 0, 0)
node1 = Node(1, 1, 0, 0)
node2 = Node(2, 0, 1, 0)
node3 = Node(3, 0, 0, 1)

ele = C3D4(0, [node0, node1, node2, node3], young, poisson, density)
matB = ele.makeBmatrix()

In [ ]:
matB

array([[-1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.],
       [-1.,  0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.],
       [-1., -1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
young = 210e9
poisson = 0.3

matD = Dmatrix(young, poisson)

In [ ]:
matD.makeDematrix()

array([[2.82692308e+11, 1.21153846e+11, 1.21153846e+11, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [1.21153846e+11, 2.82692308e+11, 1.21153846e+11, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [1.21153846e+11, 1.21153846e+11, 2.82692308e+11, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.07692308e+10,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        8.07692308e+10, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 8.07692308e+10]])

In [ ]:
matK = fem.makeKmatrix()

In [ ]:
matK.shape

(13668, 13668)

In [ ]:
print(len(nodes)*3)
print(len(nodes_head)*3)

13668
34863
